# [Advent of Code 2020](https://adventofcode.com/2020)

# The toolbox


Generalised pieces of code that either can be used in multiple questions or that simply makes understand the implementation easier.

In [13]:
import operator
import re
from functools import partial, reduce
from itertools import cycle, islice, starmap


def Input(day, line_parser=str.strip):
    "Fetch the data input from disk."
    filename = f'../data/advent2020/input{day}.txt'
    with open(filename) as fin:
        return mapt(line_parser, fin)

    
def mapt(fn, *args): 
    "Do a map, and convert the results to a tuple"
    return tuple(map(fn, *args))


def quantify(data, predictate=bool):
    "Count how many items in an iterable have predicated(item) True"
    return sum(map(predictate, data))


def nth(iterable, n, default=None):
    "Returns the nth item or a default value"
    return next(islice(iterable, n, None), default)

## [Day 1: Report Repair](https://adventofcode.com/2020/day/1)

As usual, things start off easy enough, nested looping with a computationally simple upper bound. Finding numbers that add to 2020 and then returning their multiple.

In [14]:
def find_2020(nums: [int]) -> int:
    for index, n1 in enumerate(nums):
        for n2 in nums[index:]:
            if n1 + n2 == 2020:
                return n1 * n2
    raise Exception('Unable to find solution')

In [15]:
data1 = Input(1, int)
find_2020(data1)

1006176

In [16]:
assert _ == 1006176, 'Day 1.1'

In [88]:
%% time
# part 2
def find_2020_3(nums: [int]) -> int:
    for index, n1 in enumerate(nums):
        for jindex, n2 in enumerate(nums[index+1:]):
            for n3 in nums[jindex+1:]:
                if n1 + n2 + n3 == 2020:
                    return n1 * n2 * n3
    raise Exception('Unable to find solution')

find_2020_3(data1)

CPU times: user 60.9 ms, sys: 1.96 ms, total: 62.8 ms
Wall time: 62 ms


199132160

In [18]:
assert _ == 199132160, 'Day 1.2'

# [Day 2: Password Philosophy](https://adventofcode.com/2020/day/2)

In [19]:
def is_valid_password(password: str, rule: (str, int, int)) -> bool:
    match, lower, upper = rule
    occurance = quantify(password, lambda c: c == match)
    return lower <= occurance <= upper

In [78]:
def password_and_match(line):
    chunks = re.sub('[-:]', ' ', line).strip().split(' ')
    lower, upper, match, _, password = chunks
    return password, (match, int(lower), int(upper))

data2 = Input(2, password_and_match)
quantify(starmap(is_valid_password, data2))

600

In [49]:
assert _ == 600, 'Day 2.1'

In [68]:
# part 2
def is_valid_password2(password: str, rule: (str, int, int)) -> bool:
    match, lower, upper = rule
    is_match = lambda n: password[n - 1] == match
    matches = int(is_match(lower)) + int(is_match(upper))
    return matches == 1

quantify(starmap(is_valid_password2, data2))

CPU times: user 1.2 ms, sys: 6 µs, total: 1.21 ms
Wall time: 1.22 ms


245

In [54]:
assert _ == 245, 'Day 2.2'

# [Day 3: Toboggan Trajectory](https://adventofcode.com/2020/day/3)

Here we can simply make use of some modulo arithmitic to loop back through the row when we attempt to index it out of bounds.

In [62]:
def count_trees(grid: [str], x_step: int, y_step: int):
    path = []
    for index, row in enumerate(grid[::y_step]):
        char_index = (x_step * index) % len(row)
        path.append(row[char_index])
    return quantify(path, lambda c: c == '#')
    

data3 = Input(3)
count_trees(data3, 3, 1)

151

In [44]:
assert _ == 151, 'Day 4.1'

In [59]:
# part 2
count_trees_3 = partial(count_trees, data3)
steps = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
reduce(
    operator.mul,
    starmap(count_trees_3, steps)
)

7540141059

In [40]:
assert _ == 7540141059, 'Day 3.2'

# [Day 4: Passport Processing](https://adventofcode.com/2020/day/4)

This one was a bit fiddly in terms of the regexes, I had an error thanks to not ensuring the match ended at the end of a line (i.e. missing a `$`.)

In [280]:
validators = {
    'byr': lambda x: 1920 <= int(re.match('\d{4}$', x)[0]) <= 2002,
    'iyr': lambda x: 2010 <= int(re.match('\d{4}$', x)[0]) <= 2020,
    'eyr': lambda x: 2020 <= int(re.match('\d{4}$', x)[0]) <= 2030,
    'hgt': lambda x: (
        150 <= int(re.match('(\d+)(?:cm|in)$', x)[1]) <= 193
        if re.match('\d+(cm|in)$', x)[1] == 'cm' else
        59 <= int(re.match('(\d+)(?:cm|in)$', x)[1]) <= 76
    ),
    'hcl': lambda x: bool(re.match('#[0-9a-f]{6}$', x)),
    'ecl': lambda x: bool(re.match('amb|blu|brn|gry|grn|hzl|oth$', x)),
    'pid': lambda x: bool(re.match('\d{9}$', x)),
    'cid': lambda x: True
}

def is_valid_passport(passport, validate_fields=False):
    required = set(['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'])
    if len(required - set([key for key, _ in passport])) > 0:
        return False

    if validate_fields:
        try:
            return all(
                validators[key](value)
                for key, value
                in passport
            )
        except TypeError:
            # regexes may not match, and will error if we attempt to index them
            return False
    return True


def parse_data(data):
    passports = [[]]
    for line in data:
        if not line.strip():
            # Move onto the next passport
            passports.append([])
            continue
        passports[-1].extend(
            re.findall(r'(\w+):([\w#\d]+)\b', line)
        )
        
    if not passports[-1]:
        passports.pop()
    
    return passports

In [281]:
input4 = parse_data(Input(4))
quantify(map(is_valid_passport, input4))

235

In [282]:
assert _ == 235, 'Day 4.1'

In [284]:
# part 2
is_valid_passport_strict = lambda passport: is_valid_passport(passport, True)
quantify(map(is_valid_passport_strict, input4))

194

In [238]:
assert _ == 194, 'Day 4.2'

# [Day 5: Binary Boarding](https://adventofcode.com/2020/day/5)

Today's one invovled writing a reverse binary search, rather than looking for a value in a sorted list, we're instead given the steps to perform to find our seat.

For the second part we just loop through the list of sorted seat IDs until we find a gap, the gap being our seat.


In [317]:
def reverse_binary(steps: str, lower: int, upper: int) -> int:
    step = steps[0]
    pivot = lower + (upper - lower) // 2
    if step in ('F', 'L'):
        if len(steps) == 1:
            return lower
        return reverse_binary(steps[1:], lower, pivot)
    else:
        if len(steps) == 1:
            return upper
        return reverse_binary(steps[1:], pivot + 1, upper)

def docode_boarding_pass(boarding_pass: str) -> (int, int, int):
    row = reverse_binary(boarding_pass[:7], 0, 127)
    col = reverse_binary(boarding_pass[7:], 0, 7)
    seat_id = row * 8 + col
    return (row, col, seat_id)

In [313]:
input5 = Input(5)

decoded_passes = mapt(docode_boarding_pass, input5)
seat_ids = tuple(seat_id for _, _, seat_id in decoded_passes)
max(seat_ids)

926

In [305]:
assert _ == 926, 'Day 5.1'

In [316]:
# part 2
def find_missing_seat_id(seat_ids):
    sorted_ids = sorted(seat_ids)
    prev = sorted_ids[0]

    for seat_id in sorted_ids[1:]:
        if seat_id - 2 == prev:
            return seat_id - 1
        
        prev = seat_id

find_missing_seat_id(seat_ids)

657

In [318]:
assert _ == 657, 'Day 5.2'